## Borrower Reliability Analysis 

### Client - bank credit department 
### Target of the analysis: find out whether the demographics affects the fact of repayment of the loan on time
### Data: bank statistics

The study results may be taken into account when building the model ** of credit scoring ** - a special system that evaluates the ability of a potential borrower to repay a loan to a bank.

### Data

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') #читаем файл с ресурса

In [2]:
data.info()
print()
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4 

### Вывод

Всего в таблице с данными 21525 строк и 12 столбцов.
Столбцы days_employed и total_income имеют по 19351 значений, что меньше, чем длина таблицы, есть взаимосвязь, необходимо проверить далее. Оба столбца числовые.
Типы данных по столбцам в целом соответствуют их предназначению, требуется точная проверка.
Категориальные переменные: education, family_status, gender, income_type.
Логические переменные: education_id, family_status_id, debt.
Остальные переменные количественные (purpose - возможно требуется привести к категориальной). 

### Data preprocessing

### Empty cells processing

Работа со столбцом "days_employed". Выделяется количество данных в столбце = 19351, это меньше, чем длина таблицы.
Также видно, что данные двух видов - о знаком минуса и без него. Логика подсказывает, что правильные числа - со знаком минуса, так как если разделить число без минуса на количество дней в году, что мы получаем очень большие сроки для трудового стажа одного человека. Нужно понять сколько таких значений и на что они влияют.

In [3]:
#проверяем разные условия для понимания групп и количества

days_employed_retired = data[(data['days_employed']>0) & (data['income_type']=='пенсионер')].count()
print(days_employed_retired)
print()
days_employed_jobless = data[(data['days_employed']>0) & (data['income_type']=='безработный')].count()
print(days_employed_jobless)

children            3443
days_employed       3443
dob_years           3443
education           3443
education_id        3443
family_status       3443
family_status_id    3443
gender              3443
income_type         3443
debt                3443
total_income        3443
purpose             3443
dtype: int64

children            2
days_employed       2
dob_years           2
education           2
education_id        2
family_status       2
family_status_id    2
gender              2
income_type         2
debt                2
total_income        2
purpose             2
dtype: int64


Проверка показала, что числа с минусом относятся к данным из столбца "income_type" == "пенсионер" и "безработный".
Первых 3443, вторых 2.  
Значения не меняем.
Проверяем пропуски.

Проверяем 'days_employed': NaN = 2174 строки.

In [4]:
print(data[data['days_employed'].isnull()]) # проверяем столбец на пропуски

       children  days_employed  dob_years            education  education_id  \
12            0            NaN         65              среднее             1   
26            0            NaN         41              среднее             1   
29            0            NaN         63              среднее             1   
41            0            NaN         50              среднее             1   
55            0            NaN         54              среднее             1   
65            0            NaN         21              среднее             1   
67            0            NaN         52               высшее             0   
72            1            NaN         32               высшее             0   
82            2            NaN         50               высшее             0   
83            0            NaN         52              среднее             1   
90            2            NaN         35               высшее             0   
94            1            NaN         3

In [5]:
days_employed_nan = data[(data['days_employed']==0) & (data['total_income']==0)].count() 
#проверяем разные условия для понимания групп и количества
print(days_employed_nan)

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64


Значения в столбце "days_employed" напрямую связаны со значениями столбца "total_income".
Нет значений в одном, значит нет значений и в другом столбце.

'days_employed': NaN = 2174 строки. Применяем fillna на 0.

In [6]:
data['days_employed'] = data['days_employed'].fillna(0)

In [7]:
print(data[data['days_employed'].isnull()]) # проверяем столбец на пропуски

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Столбец "days_employed" теперь 21525 значений.

Проверяем столбец "total_income" с его 19351 значением.

In [9]:
print(data[data['total_income'].isnull()]) # проверяем столбец на пропуски

       children  days_employed  dob_years            education  education_id  \
12            0            0.0         65              среднее             1   
26            0            0.0         41              среднее             1   
29            0            0.0         63              среднее             1   
41            0            0.0         50              среднее             1   
55            0            0.0         54              среднее             1   
65            0            0.0         21              среднее             1   
67            0            0.0         52               высшее             0   
72            1            0.0         32               высшее             0   
82            2            0.0         50               высшее             0   
83            0            0.0         52              среднее             1   
90            2            0.0         35               высшее             0   
94            1            0.0         3

Значения NaN в данном столбце связаны со столбцом "days_employed", где нулевые значения влияют на заполнение этого столбца.

Проверка подтвердила взаимосвязь. Заполняем NaN в "total_income" как 0. 

In [10]:
data['total_income'] = data['total_income'].fillna(0)
print(data[data['total_income'].isnull()]) # проверяем столбец на пропуски

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose]
Index: []


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Все пропуски по столбцам заполнены.

### Recap

Были обнаружены пропущенные значения в столбцах "days_employed" и "total_income" в количестве 2174 штуки.
Эти значения относятся к числовым.
Можно предположить, что одно значение рассчитывается на основании наличия другого по формуле.
Пропуски заполнены методом fillna на 0.

### Data types replacement

In [12]:
print(data.head(20)) # изучим первые 20 строк

    children  days_employed  dob_years            education  education_id  \
0          1   -8437.673028         42               высшее             0   
1          1   -4024.803754         36              среднее             1   
2          0   -5623.422610         33              Среднее             1   
3          3   -4124.747207         32              среднее             1   
4          0  340266.072047         53              среднее             1   
5          0    -926.185831         27               высшее             0   
6          0   -2879.202052         43               высшее             0   
7          0    -152.779569         50              СРЕДНЕЕ             1   
8          2   -6929.865299         35               ВЫСШЕЕ             0   
9          0   -2188.756445         41              среднее             1   
10         2   -4171.483647         36               высшее             0   
11         0    -792.701887         40              среднее             1   

При изучении выявлены артефакты в следующих столбцах:
days_employed - значения со знаком минус и слишком большие значения чтобы быть сроком;
education - разный шрифт, возможно потребуется привести к одному виду для анализа;

Проверим столбец "children" через подсчет уникальных значений. 

In [13]:
print(data['children'].value_counts()) #проверяем кол-во уникальных значений
print(data['children'].sum()) #проверяем на числа, исключаем str

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
11600


При изучении первого столбца 'children' бросаются в глаза значения не похожие на реальные: 20 и -1 ребенок на заёмщика. Данные из этого столбца являются важными для ответа на один из поставленных вопросов, попробуем разобраться детальнее.

В столбце 'children' все значения числовые.
Однако, мы видим необычные количества в 20 и в -1 детей.
Учитывая, что после 5 детей далее идет разрыв до 20, значит эти 20 детей на заемщика, это или ошибка печати вместо 2-х, либо работа системы - округление всех значений от 6 и выше в число 20. 

In [14]:
print(data[data['children'] == 20]) # выводим таблицу данных с кол-вом детей ==20, найти взаимосвязи

       children  days_employed  dob_years            education  education_id  \
606          20    -880.221113         21              среднее             1   
720          20    -855.595512         44              среднее             1   
1074         20   -3310.411598         56              среднее             1   
2510         20   -2714.161249         59               высшее             0   
2941         20   -2161.591519          0              среднее             1   
3302         20       0.000000         35              среднее             1   
3396         20       0.000000         56               высшее             0   
3671         20    -913.161503         23              среднее             1   
3697         20   -2907.910616         40              среднее             1   
3735         20    -805.044438         26               высшее             0   
3877         20   -8190.644409         45              среднее             1   
5020         20    -231.783475         3

Явные взаимосвязи с остальными переменными по другим столбцам не видны на первый взгляд.
Есть решение проверить через критическую для нас величину "наличие факта задолженности по кредиту".
Выясним соотношение в группе и решим как исправить значение в "20 детей", чтобы минимально повлиять на дальнейшие расчеты.

In [15]:
children_debt = data[(data['children']==20) & (data['debt']==1)].count() 
# найти количество заемщиков с кол-вом детей 20 и наличием факта задолженности по кредиту
print(children_debt) # найдено, что должников 8 к 76 = 10.5%

children            8
days_employed       8
dob_years           8
education           8
education_id        8
family_status       8
family_status_id    8
gender              8
income_type         8
debt                8
total_income        8
purpose             8
dtype: int64


In [16]:
count_debt = data['debt'].value_counts() # подсчитаем всего значений по столбцу debt
print(count_debt)
print(count_debt/len(data)*100) #соотношение фактов

0    19784
1     1741
Name: debt, dtype: int64
0    91.911731
1     8.088269
Name: debt, dtype: float64


Соотношение должников к общему числу заемщиков по группе с 20 детьми = 10.5%, когда как по всей базе такое соотношение 8.1%.
Логичным видится найти соотношение по всем группам, чтобы переименовать число 20 в правильную группу (с похожей характеристикой по отношению).

In [17]:
debt_grouped = data.groupby('children').agg({'debt': ['sum', 'count']}) 
# группируем по количеству детей и выводим по этим группам сумму и кол-во должников. 
debt_grouped['ratio'] = debt_grouped['debt']['sum'] / debt_grouped['debt']['count'] 
# находим соотношение суммы к количеству должников по группе #детей.
print(debt_grouped)

          debt            ratio
           sum  count          
children                       
-1           1     47  0.021277
 0        1063  14149  0.075129
 1         444   4818  0.092154
 2         194   2055  0.094404
 3          27    330  0.081818
 4           4     41  0.097561
 5           0      9  0.000000
 20          8     76  0.105263


Группа "2 ребенка" имеет близкий уровень соотношения наличия факта долга к общему количеству к группе "20 детей". Предположим, что здесь была опечатка и переименуем 20 в 2. Эта замена не сильно повлияет на дальнейшее исследование.

In [18]:
data['children'] = data['children'].replace(20, 2) # заменили 20 на 2
print(data['children'].value_counts())

 0    14149
 1     4818
 2     2131
 3      330
-1       47
 4       41
 5        9
Name: children, dtype: int64


По аналогии поступим с показателем -1 в столбце "количество детей": проверим через список и если не будет явных групп по каким-то признакам - отнесем к группе значений с похожим отношением наличие факта долга к кол-ву заемщиков.

In [19]:
print(data[data['children'] == -1]) # выводим таблицу данных с кол-вом детей ==-1, найти взаимосвязи

       children  days_employed  dob_years            education  education_id  \
291          -1   -4417.703588         46              среднее             1   
705          -1    -902.084528         50              среднее             1   
742          -1   -3174.456205         57              среднее             1   
800          -1  349987.852217         54              среднее             1   
941          -1       0.000000         57              Среднее             1   
1363         -1   -1195.264956         55              СРЕДНЕЕ             1   
1929         -1   -1461.303336         38              среднее             1   
2073         -1   -2539.761232         42              среднее             1   
3814         -1   -3045.290443         26              Среднее             1   
4201         -1    -901.101738         41              среднее             1   
4402         -1  398001.302888         64              СРЕДНЕЕ             1   
4542         -1   -1811.899756         3

Явные взаимосвязи с остальными переменными по другим столбцам не видны на первый взгляд.
Отнесем эту группу к группе "0 детей", так как это самый близкий по ratio блок.

In [20]:
data['children'] = data['children'].replace(-1, 0) # заменили -1 на 0
print(data['children'].value_counts())

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


В итоге мы сделали в столбце 'children' четкие 6 групп от 0 до 5 по кол-ву детей.

Проверил все столбцы через методы value_counts и sum:
Столбец "dob_years" - 101 значение "0" лет. Количество незначительное, на расчеты не влияет - ничего не предпринимаем.
Столбец "gender" - одно значение XNA, нужно проверить.

In [21]:
#print(data['purpose'].value_counts()) #проверяем кол-во уникальных значений
#print(data['purpose'].sum()) #проверяем на числа, исключаем str

In [22]:
print(data['gender'].value_counts())

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [23]:
print(data[data['gender'] == 'XNA']) # вбрал XNA в столбце пол.

       children  days_employed  dob_years            education  education_id  \
10701         0   -2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose  
10701  203905.157261  покупка недвижимости  


Проверка показала, что значение 'gender' == 'XNA' не является критичным в силу отсутствия влияния на дальнейшие расчеты. Оставляем как есть.

In [24]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Заменим данные в столбце "days_employed" на целочисленные в новом столбце.

In [25]:
data['days_employed_int'] = data['days_employed'].astype('int') 
# создадим новый столбец для 'days_employed_int' целых чисел
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  days_employed_int  
0               покупка жилья              -8437  
1     приобретение а

In [26]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children             21525 non-null int64
days_employed        21525 non-null float64
dob_years            21525 non-null int64
education            21525 non-null object
education_id         21525 non-null int64
family_status        21525 non-null object
family_status_id     21525 non-null int64
gender               21525 non-null object
income_type          21525 non-null object
debt                 21525 non-null int64
total_income         21525 non-null float64
purpose              21525 non-null object
days_employed_int    21525 non-null int64
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB
None


### Recap

Кроме замены нескольких нерядовых значений в столбце "children".
Заменил значение "трудовая занятость в днях" ('days_employed_int') с float в int. Переводить отрицательные значения пока нет необходимости.

### Duplicates processing

Проверим столбцы на дубликаты методами value_count и duplicated()

In [27]:
print(data['education'].value_counts()) #проверяем кол-во уникальных 
print(data.duplicated('education').sum()) # проверяем дубли

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64
21510


Дубликаты выявлены в столбцах:
'education'
Приводим к lower регистру в том же столбце.

In [28]:
data['education'] = data['education'].str.lower()

In [29]:
print(data['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


### Recap

Выявлены дубликаты в столбце "education". Устранены путем изменения регистра значений. Данные возникли возможно из-за того, что нет проверки на регистр поля на входе данных, нужно применить правило нижнего регистра по-умолчанию. Возможно также данные поступают из разных источников. В пользу этого предположения говорит единые названия значений. 

### Lemmatization

In [30]:
from pymystem3 import Mystem #загружаем библиотеку
m = Mystem()

В нашей базе присутствует один столбец, который важен для рачета, но содержит информацию, которую передавал заемщик в свободной форме: "purpose" - цель кредита.
Есть необходимость создать дополнительный столбец для дальнейшей категоризации, который будет содержать в себе ключевое слово из столбца "цель кредита".
Используем лемматизацию.

In [31]:
data['purpose_lemma'] = data['purpose'].apply(m.lemmatize) 
# добавляем столбец 'purpose_lemma' и лемматизируем в него строку из 'purpose'
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,days_employed_int,purpose_lemma
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,-8437,"[покупка, , жилье, \n]"
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,-4024,"[приобретение, , автомобиль, \n]"
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,-5623,"[покупка, , жилье, \n]"
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,-4124,"[дополнительный, , образование, \n]"
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,340266,"[сыграть, , свадьба, \n]"


Теперь необходимо purpose_lemma перевести в список категориальных значений, по которому дальше  изучать категории.

In [32]:
def get_purpose(purpose_lemma):  #функция, через которую отбираем по значениям и добавляем в новый столбец
    if 'свадьба' in purpose_lemma:
        return 'свадьба'
    if 'жильё' in purpose_lemma:
        return 'недвижимость'
    if 'жилье' in purpose_lemma:
        return 'недвижимость'
    if 'образование' in purpose_lemma:
        return 'образование'
    if 'автомобиль' in purpose_lemma:
        return 'автомобиль'
    if 'недвижимость' in purpose_lemma:
        return 'недвижимость'

    return 'other'
    
data['purpose_id'] = data['purpose_lemma'].apply(get_purpose)  # добавить новый столбец 'purpose_id' с данными из функции
#print(data[data['purpose_id'] == 'other'].head()) # через логическую функцию смотрю какие леммы еще не проверены. 

Проверяем как сложились категории по цели кредита:

In [33]:
print(data['purpose_id'].value_counts())

недвижимость    10840
автомобиль       4315
образование      4022
свадьба          2348
Name: purpose_id, dtype: int64


In [34]:
def get_purpose_business(purpose_lemma):  #функция, через которую отбираем по занчениям и добавляем в новый столбец
    if 'коммерческий' in purpose_lemma:
        return 'Yes'
    if 'сдача' in purpose_lemma:
        return 'Yes'
    if 'бизнес' in purpose_lemma:
        return 'Yes'

    return 'No'
    
data['purpose_id_business'] = data['purpose_lemma'].apply(get_purpose_business)  
# добавить новый столбец 'purpose_id_business' с данными из функции
#print(data[data['purpose_id'] == 'other'].head()) 
# через логическую функцию смотрю какие леммы еще не проверены. 

In [35]:
print(data['purpose_id_business'].value_counts())

No     19557
Yes     1968
Name: purpose_id_business, dtype: int64


Все цели кредита уложились в 4 категории: недвижимость (вкл.коммерческую), автомобиль, образование, свадьба. Из них на коммерческие цели было 1968 запросов.

### Recap

Все обращения за кредитом можно разделить на 4 блока, где "недвижимость" - самый большой (10840 значений), блок "автомобиль" - второй по значимости, но уже белее, чем в два раза меньше (4315 значений), немного уступает предыдущему блок "образование" - 4022 значений, и замыкает цепочку блок "свадьба"  - 2348 значений.

### Data categorization

Для ответа на вопросы нам потребуется категоризация: "children" - Yes/No; и "total_income" - определим шаг в процессе выполнения. Применим словари, где необходимо.

'children' - Yes/No

In [36]:
def children_group(number):
        """
        Возвращает значение Yes или No, используя правила:
        - 'Yes' при значении number !=0;
        - 'No' при значениии number ==0;
        """

        if number != 0:
                return 'Yes'
        if number == 0:
                return 'No'
            
#Протестируем работу функции для каждого правила: 
#print(children_group(5))

In [37]:
data['children_group'] = data['children'].apply(children_group) #применим функцию и добавим столбец
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  days_employed_int  \
0               покупка жилья              -8437   
1     приобретение

Категоризация по children - Yes/No применена.

In [38]:
print(data['children_group'].value_counts())

No     14196
Yes     7329
Name: children_group, dtype: int64


Разбиваем "total_income" на категории.

In [39]:
print(data[data['total_income']<0]) # проверка на наличие отрицательных значений

Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, days_employed_int, purpose_lemma, purpose_id, purpose_id_business, children_group]
Index: []


In [40]:
total_income_max = data['total_income'].max()
print(total_income_max)

2265604.028722744


Категоризация "total_income" - 0-120K, 121K-180K, 181K-240K, 241K-360K, 361K-up.

In [41]:
def income_group(number):

        if number <0:
            return 'check number'
        if number >=0:
            if number <=120000:
                return '0-120K'
        if number >121000:
            if number <=180000:
                return '121K-180K'
        if number >181000:
            if number <=240000:
                return '181K-240K'
        if number >241000:
            if number <=360000:
                return '241K-360K'
        return '361K-up'
            
#Протестируем работу функции для каждого правила: 
print(income_group(180000))

121K-180K


Присваиваем  новому столбцу 'income_group' значения, полученные по итогам работы функции.

In [42]:
data['income_group'] = data['total_income'].apply(income_group)
print(data.head())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  days_employed_int  \
0               покупка жилья              -8437   
1     приобретение

In [43]:
#группируем по новому столбцу income_group для проверки значений
print(data['income_group'].value_counts())

0-120K       9019
121K-180K    5982
181K-240K    3228
241K-360K    2310
361K-up       986
Name: income_group, dtype: int64


Категоризация по доходу выглядет так:
0-120K       9019
121K-180K    5982
181K-240K    3228
241K-360K    2310
361K-up       986

Категоризируем семейное положение заёмщика.

In [44]:
family_status_log = data[['family_status_id', 'children']] #категоризация статуса id и количества детей исходя из статуса
print(family_status_log.head())

   family_status_id  children
0                 0         1
1                 0         1
2                 0         0
3                 0         3
4                 1         0


In [45]:
family_status_dict = data[['family_status_id', 'family_status']]  #словарь через id семейного статуса и название статуса
print(family_status_dict.head(10))

   family_status_id     family_status
0                 0   женат / замужем
1                 0   женат / замужем
2                 0   женат / замужем
3                 0   женат / замужем
4                 1  гражданский брак
5                 1  гражданский брак
6                 0   женат / замужем
7                 0   женат / замужем
8                 1  гражданский брак
9                 0   женат / замужем


In [46]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)  # удаляем дубликаты в словаре
print(family_status_dict.head())

   family_status_id          family_status
0                 0        женат / замужем
1                 1       гражданский брак
2                 2         вдовец / вдова
3                 3              в разводе
4                 4  Не женат / не замужем


In [47]:
print(family_status_log.groupby('family_status_id').mean().sort_values('children',ascending=False).head(10)) 
# группируем по статусу и выводим среднее количество детей на заемщика

                  children
family_status_id          
0                 0.569305
1                 0.459660
3                 0.430962
4                 0.230715
2                 0.153125


### Recap

Категоризировал столбцы "children", где  значений No = 14196 и Yes = 7329;
и "total_income" (ежемесячный доход), где 0-120K - 9019 наблюдений; 121K-180K - 5982; 181K-240K - 3228; 241K-360K - 2310; 361K-up - 986; Также сделал категоризацию через id семейного статуса и вывел среднее количество детей на каждую категорию.

### Hypotheses

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [48]:
data.pivot_table(values='debt',index='children_group',aggfunc=['count','mean']) 
# пивот где показать среднюю задолженность исходя из наличия детей Yes/No

,count,mean
,debt,debt
children_group,,
No,14196,0.074951
Yes,7329,0.092373


In [49]:
data.pivot_table(values='debt',index='children',aggfunc=['count','mean'])
# проверим средние значения применяя группировку по кол-ву детей

,count,mean
,debt,debt
children,,
0,14196,0.074951
1,4818,0.092154
2,2131,0.094791
3,330,0.081818
4,41,0.097561
5,9,0.000000


### Recap

Зависимость между наличием детей и возвратом кредита в срок есть.
Группа заёмщиков с детьми любого количества имеет величину на 1,74 п.п. выше, чем без детей (9,2% и 7,5% средние значения соответственно).
Из групп заёмщиков с детьми верхние два значения  у заёмщиков с 4-мя и 2-мя детьми (9,76% и 9,48% средние значения соответственно).
Минимальное среднее значение группы заёмщиков с детьми у тех, кто указал наличие 3-х детей (8,2%).
Для кредитного скоринга можно оставить зависимость от деления на две группы с и без детей, так как деление по количеству детей не даёт большой разницы в средних значениях.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [50]:
data.pivot_table(values='debt',index='family_status',aggfunc=['count','mean'])
# пивот где показать среднюю задолженность исходя из данных по семейному статусу

,count,mean
,debt,debt
family_status,,
Не женат / не замужем,2813,0.097405
в разводе,1195,0.071130
вдовец / вдова,960,0.065625
гражданский брак,4177,0.092890
женат / замужем,12380,0.075202


In [51]:
data.pivot_table(values='debt',index='family_status',columns='gender',aggfunc=['count','mean'])
# добавим значения по гендерному признаку

count                   mean               
gender                      F       M  XNA         F         M  XNA
family_status                                                      
Не женат / не замужем  1732.0  1081.0  NaN  0.068129  0.144311  NaN
в разводе               936.0   259.0  NaN  0.065171  0.092664  NaN
вдовец / вдова          905.0    55.0  NaN  0.057459  0.200000  NaN
гражданский брак       2868.0  1308.0  1.0  0.081241  0.118502  0.0
женат / замужем        7795.0  4585.0  NaN  0.067992  0.087459  NaN

### Recap

Зависимость  между данными по семейному положению и возвратом кредита в срок есть.
Самая низкая дисциплина у заёмщиков со статусом "не женат / не замужем" - в среднем 9,7% случаев нарушения сроков оплаты кредита. Причём нужно отметить, что такой высокий процент нарушений обеспечивают лица мужского пола - 14,4% с нарушениями, когда как средний % нарушений у заёмщиков женского пола значительно ниже, чем по группе в целом - 6,8% (группа 9,7%).
Вторая значимая группа по количеству нарушений кроков кредита "гражданский брак" - 9,3%. Аналогичная картина с разбивкой по гендерному признаку - доля нарушений женщинами-заёмщиками ниже средних значений группы.
Самая лучшая ситуация с наличием просроченных кредитов в группе "вдовец/вдова" - 6,6%. Но, нужно учесть общую тенденцию низкой дисциплины у заёмщиков мужского пола - 20%.
Исходя из вышесказанного, в кредитном скоринге необходимо учесть гендерную составляющую, а не только данные по тому или иному семейному статусу.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
print(data.head())
print()
print(data['total_income'].mean())
print()
print(data['total_income'].median())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0      M   сотрудник     0  145885.952297   
3   женат / замужем                 0      M   сотрудник     0  267628.550329   
4  гражданский брак                 1      F   пенсионер     0  158616.077870   

                      purpose  days_employed_int  \
0               покупка жилья              -8437   
1     приобретение

In [53]:
data.pivot_table(values='debt',index='income_group',aggfunc=['count','mean'])
# пивот где показать среднюю задолженность исходя из данных по доходу заёмщика

,count,mean
,debt,debt
income_group,,
0-120K,9019,0.079942
121K-180K,5982,0.088599
181K-240K,3228,0.079926
241K-360K,2310,0.071861
361K-up,986,0.066937


In [54]:
data.groupby(['income_type','income_group'])['debt'].agg(['count','mean']).sort_values('mean',ascending=False)
# сгруппируем по группам дохода и изучим самые рискованные группы по типу занятости 'income type'

count      mean
income_type     income_group                 
безработный     0-120K            1  1.000000
в декрете       0-120K            1  1.000000
сотрудник       121K-180K      3260  0.106135
                0-120K         4726  0.094160
                181K-240K      1638  0.089744
                241K-360K      1089  0.083563
компаньон       121K-180K      1380  0.081159
сотрудник       361K-up         406  0.078818
компаньон       0-120K         1534  0.075619
                181K-240K       957  0.073145
                361K-up         396  0.070707
госслужащий     241K-360K       164  0.067073
пенсионер       181K-240K       411  0.065693
госслужащий     0-120K          598  0.065217
                181K-240K       221  0.063348
компаньон       241K-360K       818  0.061125
пенсионер       241K-360K       239  0.058577
                0-120K         2157  0.055169
                121K-180K       945  0.053968
госслужащий     121K-180K       397  0.052897
пенсионер       361K-up         104  0.048077
госслужащий     361K-up          79  0.012658
предприниматель 0-120K            1  0.000000
                361K-up           1  0.000000
безработный     181K-240K         1  0.000000
студент         0-120K            1  0.000000

In [55]:
data.groupby(['income_group','education'])['debt'].agg(['count','mean']).sort_values('mean',ascending=False)
# категоризация по уровню дохода и уровню образования

,,count,mean
income_group,education,,
361K-up,неоконченное высшее,38,0.184211
121K-180K,начальное,70,0.142857
181K-240K,начальное,36,0.138889
241K-360K,неоконченное высшее,115,0.113043
181K-240K,неоконченное высшее,126,0.111111
121K-180K,среднее,4318,0.096341
0-120K,начальное,157,0.095541
361K-up,среднее,451,0.088692
181K-240K,среднее,2123,0.087141


### Recap

Зависимость между данными по уровню дохода заёмщика и возвратом кредита в срок есть. Однако различия между группами по доходам по среднему уровню нарушения кредитной дисциплины невелики: от 6,7% (доход свыше 351000 в месяц) до 8,86% (от 121000 до 180000 рублей в месяц). 
Если применить группировку по типу занятости, то мы видим, что есть большие группы заёмщиков, которые имеют величину выше, чем по группам дохода: 9624 заёмщика (45% базы) из группы "сотрудник" с доходом от минимума до 240000 рублей в месяц имеют величины от 8,97% до 10,6% случаев нарушения (максимум по группам дохода 8,86%). Остальные группы по типу занятости показывают лучшие результаты. 
Углубляясь в детали можно найти статистику по уровню нарушений кредитной дисциплины исходя из уровня образования заёмщика: 7 групп из 23 (30%) имеют превышение по средним значениям вплоть до 18,4%. 
Исходя из вышесказанного можно рекомендовать применить скоринг не сколько к категориям по уровню дохода отдельно, сколько в связке с уровнем образования и типом занятости.

- Как разные цели кредита влияют на его возврат в срок?

In [56]:
data.groupby(['purpose_id'])['debt'].agg(['count','mean']).sort_values('mean',ascending=False)
# категоризация по цели кредита и факта долга

,count,mean
purpose_id,,
автомобиль,4315,0.093395
образование,4022,0.091994
свадьба,2348,0.079216
недвижимость,10840,0.072140


In [57]:
data.groupby(['purpose_id_business','purpose_id'])['debt'].agg(['count','mean']).sort_values('mean',ascending=False)
# категоризация по цели кредита и факта долга

count      mean
purpose_id_business purpose_id                   
No                  автомобиль     4315  0.093395
                    образование    4022  0.091994
                    свадьба        2348  0.079216
Yes                 недвижимость   1968  0.076728
No                  недвижимость   8872  0.071123

### Recap

По целям кредита можно сказать, что самый рискованный вид цели - "автомобиль" (9,3% фактов просроченного кредита); второй - "образование" с 9,2%. С большим отрывом идет "свадьба" - 7,9% и замыкает "недвижимость" с 7,2% нарушений.
Деление на коммерческую и некоммерческую недвижимость не даёт большой разницы в значениях - нет причин выделять в отдельную категорию ввиду количества.

### Output

Изучение данных показало, что присутствуют такие группы заёмщиков, по которым количество фактов просроченных кредитов достигает высоких значений (до 19%). Речь идет о таких характеристиках как пол, уровень образования.
Если применять скоринг к группам, поделенным по базовому принципу, как в поставленных вопросах, то к значительному различию в значениях это не приводит: вариации в 2-3 п.п. 